In [ ]:
import pandas as pd
import os
import numpy as np
import time

from BPt import *
from BPt.extensions import SurfLabels, SurfMaps
from sklearn.linear_model import ElasticNetCV

import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
param_search = Param_Search(search_type='RandomSearch',
                            n_iter=60,
                            splits=3,
                            n_repeats=1,
                            cv=CV(groups='rel_family_id'))

elastic = Model('elastic', params=1,
                param_search=param_search,
                extra_params={'tol': 1e-3})

In [ ]:
ML = Load('../data/Base_consol.ML')
ML.log_dr = None
ML.n_jobs = 16
parcel = 'vol-resamp-difumo-1024_prob'


rois = SurfMaps(maps = '../parcels/' + parcel + '.npy', strategy='ls')
loader = Loader(rois, cache_loc='/home/sage/cache/'+parcel)
pipeline = Model_Pipeline(imputers = None,
                          loaders = loader,
                          scalers = Scaler('robust'),
                          model = elastic)

results = ML.Evaluate(pipeline,
                      Problem_Spec(target=1),
                      splits=5,
                      n_repeats=1,
                      cv=CV(groups='rel_family_id'))


rois = SurfMaps(maps = '../parcels/' + parcel + '.npy', strategy='average')
loader = Loader(rois, cache_loc='/home/sage/cache/'+parcel+'_average')
pipeline = Model_Pipeline(imputers = None,
                          loaders = loader,
                          scalers = Scaler('robust'),
                          model = elastic)

results = ML.Evaluate(pipeline,
                      Problem_Spec(target=1),
                      splits=5,
                      n_repeats=1,
                      cv=CV(groups='rel_family_id'))

In [ ]:
stop

64, ls - 0.1104507335298468
64, wa - 0.1081314501079412

128, ls - 0.1456198931372817
128, wa - 0.1468823524777711

256, ls - 0.1581170657384047
256, wa - 0.1583927824602895

512, ls - 0.15258950415716147
512, wa - 0.1644101551534995


In [ ]:

i = 9, 94 for 256

In [ ]:
def get_p_model(parcel):
    
    param_search = Param_Search(search_type='RandomSearch',
                                n_iter=60,
                                splits=3,
                                n_repeats=1,
                                cv=CV(groups='rel_family_id'))

    elastic = Model('elastic', params=1,
                    param_search=param_search,
                    extra_params={'tol': 1e-3})

    rois = SurfLabels(labels = 'parcels/' + parcel + '.npy')
    loader = Loader(rois, cache_loc='/home/sage/cache/'+parcel)

    pipeline = Model_Pipeline(imputers = None,
                              loaders = loader,
                              scalers = Scaler('robust'),
                              model = elastic)

    return Model(pipeline)

In [ ]:
ML = Load('data/Base.ML')
ML.n_jobs = 16

stack_param_search = Param_Search(search_type='RandomSearch',
                                  n_iter=60,
                                  splits=3,
                                  n_repeats=1)
stack_model = Model('elastic', params=1, param_search=stack_param_search)

stack_splits = CV_Splits(cv=CV(groups='rel_family_id'), splits=5, n_repeats=1)

p1 = get_p_model('random_1000_0')
p2 = get_p_model('random_1000_1')
p3 = get_p_model('random_1000_2')
p4 = get_p_model('random_1000_3')
p5 = get_p_model('random_1000_4')

stacking_ensemble = Ensemble(obj = "stacking regressor",
                             models = [p1, p2, p3, p4, p5],
                             cv_splits = stack_splits,
                             base_model = stack_model,
                             n_jobs_type = 'models')

pipeline = Model_Pipeline(imputers=None,
                          model=stacking_ensemble)

In [ ]:
results = ML.Evaluate(pipeline, Problem_Spec(target=1), splits=5, n_repeats=1, cv=CV(groups='rel_family_id'))